## #3.0 LLMs and Chat Models

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

# ✅ LLM (기본 모델은 더 이상 사용하지 않으므로 명시적으로 지정)
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)
# llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)

# ✅ Chat 모델도 명시적으로 모델 이름 지정
# chat = ChatOpenAI(model_name="gpt-4o", temperature=0)
chat = ChatOpenAI(model_name="gpt-4o", temperature=0) # temperature은 모델의 창의성을 결정하는 숫자이다.

# 요청
a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

a, b


('\n\nAs of 2021, there are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, there may be more planets in other solar systems beyond our own.',
 'As of now, there are eight recognized planets in our solar system. These are, in order from the Sun: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Pluto was previously considered the ninth planet but was reclassified as a "dwarf planet" by the International Astronomical Union in 2006.')

## #3.1 Predict Messages

In [10]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1
)

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="you are a geography expert. And you only reply in Italian.",
    ),
    AIMessage(content = "Ciao, mi chiamo Paolo!"),
    HumanMessage(content="what is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)

AIMessage(content='La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Mi chiamo Paolo, sono un esperto di geografia. Avete altre domande?')

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="you are a geography expert. And you only reply in {language}.",
    ),
    AIMessage(content = "Ciao, mi chiamo {name}!"),
    HumanMessage(content="what is the distance between {country_a} and {country_b}. Also, what is your name?")
]
# 이런식으로 변수처럼 만들어서 사용할 수도 있다.
chat.predict_messages(messages)

## #3.2 Prompt Tempaltes

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template(
    "what is the distance between {country_a} and {country_b}"
    )

prompt = template.format(
    country_a = "Mexico",
    country_b = "Thailand"
)

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [17]:
template = ChatPromptTemplate.from_messages(
    [
    ("system","you are a geography expert. And you only reply in {language}."),
    ("ai", "ciao, mi chiamo {name}!"),
    ("human","what is the distance between {country_a} and {country_b}. Also, what is your name?")
    ]
)
# 이런식으로 변수처럼 만들어서 사용할 수도 있다.
chat.predict_messages(messages)

prompt = template.format_messages(
    language = "Greek",
    name = "Socrates",
    country_a = "Mexico",
    country_b = "Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης.')

## #3.3 OutputParser and LCEL

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputparser(BaseOutputParser):
    #outputPaser는 paser라는 메서드를 꼭 구현해야한다.
    def parse(self, text):
        return text.strip().split("'")
    
p = CommaOutputparser()

p.parse("Hello, how, are, you")
#이렇게 하면 공백이 같이 출력되어나온다.

['Hello, how, are, you']

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputparser(BaseOutputParser):
    #outputPaser는 paser라는 메서드를 꼭 구현해야한다.
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip,items))
    
p = CommaOutputparser()

p.parse("Hello, how, are, you")
#공백이 사라지게된다.

['Hello', 'how', 'are', 'you']

In [26]:
template = ChatPromptTemplate.from_messages(
        [
            ("system","you are a list generating machine. Everything you are asked will be answered with a"
        "list of max {max_items}. Do NOT reply with anything else."),
        ("human", "{question}")
    ]
)

prompt = template.format_messages(
    max_items = 10,
    question = "what are the planets?"
)

chat.predict_messages(prompt)

AIMessage(content='1. Mercury\n2. Venus\n3. Earth\n4. Mars\n5. Jupiter\n6. Saturn\n7. Uranus\n8. Neptune\n9. Pluto')

In [ ]:
template = ChatPromptTemplate.from_messages(
        [
            ("system","you are a list generating machine. Everything you are asked will be answered with a comma separated"
        "list of max {max_items}. Do NOT reply with anything else."), #comma 라고 알려주기
        ("human", "{question}")
    ]
)

prompt = template.format_messages(
    max_items = 10,
    question = "what are the planets?"
)

chat.predict_messages(prompt)

AIMessage(content='Mercury, Venus, Earth')

In [ ]:
template = ChatPromptTemplate.from_messages(
        [
            ("system","you are a list generating machine. Everything you are asked will be answered with a comma separated"
        "list of max {max_items}. Do NOT reply with anything else."), #comma 라고 알려주기
        ("human", "{question}")
    ]
)

prompt = template.format_messages(
    max_items = 10,
    question = "what are the planets?"
)

result = chat.predict_messages(prompt)

p = CommaOutputparser()
p.parse(result.content) #LLM으로 부터 성공적인 응답을 받음

['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

#### 코드가 엄청길어지는데 쉬운 방법과 사용할 LangChain Expression Language언어가 얼마나 효율적인지 살펴보기
#### 필요한것은 template와 OutputPaser 그리고 Chat model이 전부다

In [ ]:
template = ChatPromptTemplate.from_messages(
        [
            ("system","you are a list generating machine. Everything you are asked will be answered with a comma separated"
        "list of max {max_items}. Do NOT reply with anything else."), #comma 라고 알려주기
        ("human", "{question}")
    ]
)

In [30]:
chain = template |chat | CommaOutputparser() # chain을 만듬 | 연산자를 사용해서 

chain.invoke({
    "max_items":5,
    "question":"what are the pokemons?"
})

['Pikachu', 'Charmander', 'Bulbasaur', 'Squirtle', 'Jigglypuff']

In [ ]:
chain_one = template |chat | CommaOutputparser() # chain을 만듬 | 연산자를 사용해서 

# chain_two = template |chat | outputParser

# all = chain_one | chain_two | output <--- 이런식으로 chain을 만들어줄 수 있다.

chain.invoke({
    "max_items":5,
    "question":"what are the pokemons?"
})

## #3.4 Chaining Chains

In [36]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1,#streaming 활성화 streaming은 우리가 model의 응답이 생성되는걸 볼수있게 해준다.
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()] #이렇게 하면 다음번에 질문할 때는 console에서 응답의 진행을 볼 수 있다.
                  ) 

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a world-class international chef. You create easy to follow"
    "recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

In [38]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian."
    "You find alternative ingredients and explain their preparation. You don't radically modify the recipe."
    "if there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe":chef_chain}| veg_chain

final_chain.invoke({
    "cuisine" : "indian"
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's make a classic and delicious dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 can (14 oz) tomato sauce
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- 1/2 teaspoon cayenne pepper (adjust to taste)
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (for serving)

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the chicken pieces and mix well to coat. Cover and marinate in the refrigerat

AIMessageChunk(content="As a vegetarian chef, I can help you make a vegetarian version of Chicken Tikka Masala by replacing the chicken with a suitable alternative. Here's how you can modify the recipe:\n\nIngredients:\n- 1 lb of paneer or extra-firm tofu, cut into bite-sized pieces: Paneer is a popular Indian cheese that holds its shape well when cooked, making it a great substitute for chicken in this dish. Extra-firm tofu is another excellent option that can absorb the flavors of the marinade and sauce.\n- 1 cup plain yogurt: You can use plant-based yogurt made from soy, almond, or coconut milk as a dairy-free alternative.\n- Instead of lemon juice, you can use lime juice for a similar tangy flavor.\n- Use a high-heat oil like avocado oil or coconut oil instead of vegetable oil for marinating and cooking.\n- For garnish, you can use fresh cilantro as suggested in the recipe.\n\nInstructions:\n1. Follow the same marinating process as in the original recipe, but use paneer or tofu ins

## #3.5 Recap

s